In [37]:
import numpy as np
from itertools import combinations
from Flaskapp.custom_classes import Predictor

In [2]:
model = Predictor.load('models/classrng')
model

In [3]:
penc = model._penc
penc

In [4]:
enc = penc._encodings
enc

{'Player 3': 0,
 'Player 9': 1,
 'Player 17': 2,
 'Player 16': 3,
 'Player 6': 4,
 'Player 14': 5,
 'Player 1': 6,
 'Player 4': 7,
 'Player 2': 8,
 'Player 5': 9,
 'Player 11': 10,
 'Player 7': 11,
 'Player 10': 12,
 'Player 15': 13,
 'Player 18': 14,
 'Player 12': 15,
 'Player 13': 16,
 'Player 8': 17,
 'Player 19': 18,
 'Player 20': 19}

In [5]:
len(enc)

20

20 nCr 2 -> 190 impostor combinations  
After taking out the two impostors, for each of those combinations there are a further  
18 nCr 8 -> 43,758 crewmate combinations  
So with 20 players there should be 8,314,020 possible player/role combinations. That's definitely far too many to make predictions for all of them

Will numpy's choice function work with this generator? -- **No**

In [6]:
rng = np.random.default_rng()
rng.choice(list(combinations(enc.keys(), 2)), 3)

array([['Player 17', 'Player 5'],
       ['Player 2', 'Player 18'],
       ['Player 9', 'Player 15']], dtype='<U9')

What's a reasonable number of matches to use for generating the leaderboard? 10k? 100k?  
Perhaps 100 impostor combinations, with 1000 crewmate combinations for each impostor pair?

In [7]:
rng = np.random.default_rng()
plist = list(enc.keys())
# Choice can't choose more items than there are available
combs = list(combinations(plist, 2))
impostors = rng.choice(combs, min(len(combs), 100), replace=False)
matches = []
for imp in impostors:
    candidates = np.setdiff1d(plist, imp)
    ccombs = list(combinations(candidates, 8))
    crewmates = rng.choice(ccombs, min(len(ccombs), 1000), replace=False)
    matches.append({
        'impostors': imp,
        'crewmates': crewmates
    })
matches[0]

{'impostors': array(['Player 2', 'Player 8'], dtype='<U9'),
 'crewmates': array([['Player 1', 'Player 13', 'Player 14', ..., 'Player 18',
         'Player 6', 'Player 7'],
        ['Player 11', 'Player 15', 'Player 17', ..., 'Player 3',
         'Player 7', 'Player 9'],
        ['Player 1', 'Player 12', 'Player 14', ..., 'Player 5',
         'Player 6', 'Player 9'],
        ...,
        ['Player 14', 'Player 15', 'Player 16', ..., 'Player 6',
         'Player 7', 'Player 9'],
        ['Player 1', 'Player 10', 'Player 15', ..., 'Player 3',
         'Player 5', 'Player 7'],
        ['Player 10', 'Player 11', 'Player 13', ..., 'Player 4',
         'Player 5', 'Player 6']], dtype='<U9')}

That did take some time, but it could definitely be worse. I'll run with these for now

In [8]:
m = model._model
m

In [9]:
format_data = model._Predictor__convert_match_pred
match_settings = {
    'crewmates': matches[0]['crewmates'][0],
    'impostors': matches[0]['impostors'],
    'map': "Airship",
    'confirm_ejects': False,
    'emergency_meetings': 1,
    'emergency_cooldown': 20,
    'discussion_time': 15,
    'voting_time': 120,
    'anonymous_votes': True,
    'player_speed': 1.25,
    'crewmate_vision': 0.75,
    'impostor_vision': 1.5,
    'kill_cooldown': 35.0,
    'kill_distance': "Short",
    'visual_tasks': False,
    'task_bar_updates': "Meetings",
    'common_tasks': 2,
    'long_tasks': 3,
    'short_tasks': 5
}
format_data(match_settings)

array([  0.  ,   0.  ,   0.  ,   1.  ,   1.  ,   1.  ,   1.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   1.  ,   1.  ,   0.  ,
         1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,
         1.  ,   0.  ,   0.  ,   0.  ,   8.  ,   2.  ,   0.  ,   1.  ,
        20.  ,  15.  , 120.  ,   1.  ,   1.25,   0.75,   1.5 ,  35.  ,
         1.  ,   0.  ,   1.  ,   2.  ,   3.  ,   5.  ])

In [10]:
m.predict(np.array([format_data(match_settings)]))

array([[0.15519747, 0.84480256]], dtype=float32)

In [20]:
def get_leaders(settings):
    '''
    Gets the ranking of all known players for given game settings

    Required settings are:
        * map
        * player_count
        * impostor_count
        * confirm_ejects
        * emergency_meetings
        * emergency_cooldown
        * discussion_time
        * voting_time
        * anonymous_votes
        * player_speed
        * crewmate_vision
        * impostor_vision
        * kill_cooldown
        * kill_distance
        * visual_tasks
        * task_bar_updates
        * common_tasks
        * long_tasks
        * short_tasks
    '''
    settings_list = [
        'map', 'confirm_ejects', 'emergency_meetings',
        'emergency_cooldown', 'discussion_time', 'voting_time',
        'anonymous_votes', 'player_speed', 'crewmate_vision',
        'impostor_vision', 'kill_cooldown', 'kill_distance',
        'visual_tasks', 'task_bar_updates', 'common_tasks',
        'long_tasks', 'short_tasks'
    ]
    rng = np.random.default_rng()
    plist = list(enc.keys())
    # Choice can't choose more items than there are available
    combs = list(combinations(plist, 2))
    impostors = rng.choice(combs, min(len(combs), 100), replace=False)
    matches = []
    for imp in impostors:
        candidates = np.setdiff1d(plist, imp)
        ccombs = list(combinations(candidates, 8))
        crewmates = rng.choice(ccombs, min(len(ccombs), 1000), replace=False)
        limp = list(imp)
        for crew in crewmates:
            matches.append({
                'impostors': limp,
                'crewmates': list(crew),
                **{s: settings[s] for s in settings_list}
            })
    return matches

leader_matches = get_leaders(match_settings)
leader_matches[:2]

[{'impostors': ['Player 6', 'Player 12'],
  'crewmates': ['Player 10',
   'Player 11',
   'Player 13',
   'Player 17',
   'Player 3',
   'Player 4',
   'Player 7',
   'Player 8'],
  'map': 'Airship',
  'confirm_ejects': False,
  'emergency_meetings': 1,
  'emergency_cooldown': 20,
  'discussion_time': 15,
  'voting_time': 120,
  'anonymous_votes': True,
  'player_speed': 1.25,
  'crewmate_vision': 0.75,
  'impostor_vision': 1.5,
  'kill_cooldown': 35.0,
  'kill_distance': 'Short',
  'visual_tasks': False,
  'task_bar_updates': 'Meetings',
  'common_tasks': 2,
  'long_tasks': 3,
  'short_tasks': 5},
 {'impostors': ['Player 6', 'Player 12'],
  'crewmates': ['Player 10',
   'Player 13',
   'Player 17',
   'Player 18',
   'Player 19',
   'Player 20',
   'Player 5',
   'Player 9'],
  'map': 'Airship',
  'confirm_ejects': False,
  'emergency_meetings': 1,
  'emergency_cooldown': 20,
  'discussion_time': 15,
  'voting_time': 120,
  'anonymous_votes': True,
  'player_speed': 1.25,
  'crewmate_

In [22]:
results = m.predict(np.array([format_data(mch) for mch in leader_matches]))
results[:2]

array([[0.10026716, 0.8997328 ],
       [0.19099031, 0.80900973]], dtype=float32)

Now that some example matches are paired with the expected winners, sums can be taken and a leaderboard constructed.

In [32]:
wins = {p:0 for p in enc.keys()}
wenc = model._wenc
for match,result in zip(leader_matches, results):
    winner = wenc.inverse_transform([np.round(result)])[0,0]
    for p in match[winner]:
        wins[p] += 1
wins

{'Player 3': 10000,
 'Player 9': 11000,
 'Player 17': 9000,
 'Player 16': 9000,
 'Player 6': 10000,
 'Player 14': 11000,
 'Player 1': 9000,
 'Player 4': 10000,
 'Player 2': 13000,
 'Player 5': 10000,
 'Player 11': 13000,
 'Player 7': 8000,
 'Player 10': 9000,
 'Player 15': 8000,
 'Player 18': 10000,
 'Player 12': 9000,
 'Player 13': 13000,
 'Player 8': 9000,
 'Player 19': 10000,
 'Player 20': 9000}

In [36]:
sorted(wins.items(), key=lambda x: x[1], reverse=True)

[('Player 2', 13000),
 ('Player 11', 13000),
 ('Player 13', 13000),
 ('Player 9', 11000),
 ('Player 14', 11000),
 ('Player 3', 10000),
 ('Player 6', 10000),
 ('Player 4', 10000),
 ('Player 5', 10000),
 ('Player 18', 10000),
 ('Player 19', 10000),
 ('Player 17', 9000),
 ('Player 16', 9000),
 ('Player 1', 9000),
 ('Player 10', 9000),
 ('Player 12', 9000),
 ('Player 8', 9000),
 ('Player 20', 9000),
 ('Player 7', 8000),
 ('Player 15', 8000)]

In reality, the underlying player order has higher player numbers at a higher skill than lower player numbers. That's not totally reflected here, but that could easily be an issue with my data rather than the system as a whole. This is something that at least looks like a leaderboard, so I'm happy with keeping it like this